# Single Cycle CPU Assignment
## This Jupyter notebook will run the design on FPGA and display the maximum frequency of the design

In [1]:
from pynq import Overlay, Clocks

In [2]:
# Function to write instructions to registers
def write(idata):
    f = open(idata, "r")
    for count, line in enumerate(f.readlines()):
        cpu_ip.write(int(count*4),int(line, 16))
    

# Function to read RISCV 32 Registers
def read():
    out = []
    for i in range(32,64):
        out.append(cpu_ip.read(int(i*4)))
    return out


In [3]:
clocks = [50, 100, 125, 142, 200]
for i in range(0,4):
    print("Testing "+"idata"+str(i+1)+".txt")
    f = open("expout"+str(i+1)+".txt", "r")        # Expected output file
    out_crct = []
    for line in f.readlines():
        out_crct.append(int(line))
    for clock in clocks:
        ol_cpu = Overlay('overlay/harness_axi.bit')    # Programming the bitstream onto the FPGA
        cpu_ip = ol_cpu.harness_axi_ip_v1_0_0          # RISCV CPU Verilog IP
        Clocks.fclk1_mhz = clock                       # Set clock frequency in MHz
        cpu_ip.write(int(64*4),1)                      # Assign reset to 1
        write("idata"+str(i+1)+".txt")                 # Write instructions
        cpu_ip.write(int(64*4),1)                      # Assign reset to 1
        cpu_ip.write(int(64*4),0)                      # Assign reset to 0
        out = read()                                   # Read registers
        
        
        # Increase the clock frequency untill timing violations occur
        if out==out_crct:                              # Check if expected and obtained register values
            print("\t", "Frequency at", clock, "MHz: Passed")
            clock += 50
        else:
            print("\t", "Frequency at", clock, "MHz: Not passed")
#             Use below lines for debug
#             print("Expected result: ", out_crct)
#             print("Obtained result: ", out)
    print("\n")


Testing idata1.txt
	 Frequency at 50 MHz: Passed
	 Frequency at 100 MHz: Passed
	 Frequency at 125 MHz: Passed
	 Frequency at 142 MHz: Passed
	 Frequency at 200 MHz: Not passed
Expected result:  [0, 7, 4, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 1]
Obtained result:  [0, 6, 4, 2052, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0]


Testing idata2.txt
	 Frequency at 50 MHz: Passed
	 Frequency at 100 MHz: Passed
	 Frequency at 125 MHz: Passed
	 Frequency at 142 MHz: Passed
	 Frequency at 200 MHz: Not passed
Expected result:  [0, 81920, 20, 16400, 28, 28, 44, 44, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0]
Obtained result:  [0, 16384, 5, 16400, 28, 28, 12, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 0]


Testing idata3.txt
	 Frequency at 50 MHz: Passed
	 Frequency at 100 MHz

## Optional -  Run your own testcases

In the snippet below replace `test.s` (line 3) with the name of the file containing your assembly program

In [5]:
import os
from riscv_assembler.convert import AssemblyConverter
cnv = AssemblyConverter(output_type="tp", hexMode=True)
assembly_prog_name="test.s"  # --- Replace with the name of your assembly file 
top_name=assembly_prog_name.split(".")[0]
os.system("rm -rf "+top_name+"/txt/")
cnv.convert(assembly_prog_name)

with open(top_name+"/txt/"+top_name+".txt",'r') as f:
    stripped=[]
    for line in f:
        stripped.append(line[2:-1])

with open(top_name+"/txt/"+top_name+".txt",'w') as f:
    for item in stripped:
        f.write(item+"\n")

print(stripped)

------Writing to Text file------
Output file: test.txt
------Printing Output------
0x00000533
0x00050733
0x00a50613
0x000506b3
0x00e68733
0x00168693
0x7cc6cbe3
0x00070533
0x8027b3ef
Number of instructions: 9
['00000533', '00050733', '00a50613', '000506b3', '00e68733', '00168693', '7cc6cbe3', '00070533', '8027b3ef']


In [7]:
# Running assembly on CPU

ol_cpu = Overlay('overlay/harness_axi.bit')    # Programming the bitstream onto the FPGA
cpu_ip = ol_cpu.harness_axi_ip_v1_0_0          # RISCV CPU Verilog IP
Clocks.fclk1_mhz = 50                          # Set clock frequency in MHz
cpu_ip.write(int(64*4),1)                      # Assign reset to 1
write("test/txt/test.txt")                     # Write instructions
cpu_ip.write(int(64*4),1)                      # Assign reset to 1
cpu_ip.write(int(64*4),0)                      # Assign reset to 0
out = read()                                   # Read registers
print(out)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 11, 10, 1, 0, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
